In [85]:
import sys, os, datetime, dotenv, time, re, json
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from transformers import AutoTokenizer, AutoModelForCausalLM
import string
import nltk
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
import faiss
import yt_dlp

dotenv.load_dotenv(dotenv_path="../.env")
TRANSCRIPT_DIR = "transcripts"
CONFIG_DIR = "config"
## open json file 
with open(f"{CONFIG_DIR}/videos.json", "r") as f:
    video_links = json.load(f)


In [83]:
def download_transcript(id: str, filename: str, overwrite: bool) -> None:

    if not overwrite and os.path.exists(filename):
        print(f"Transcript {filename} already downloaded")
        return

    transcript = YouTubeTranscriptApi.get_transcript(
        id, languages=('en', 'en-US', 'en-GB'))
    global_time = 0
    print(f"Downloading transcript for {id} to {filename}")
    with open(filename, "w") as f:
        for line in transcript:
            new_line = ""
            # add timestamp
            new_line += f"[[{global_time}]]"
            new_line += line['text']
            f.write(f'{new_line}\n')
            global_time += float(line['duration'])
            global_time = round(global_time, 2)
    with open(f"{filename[:-4]}_clean.txt", "w") as f:
        for line in transcript:
            f.write(f'{line["text"]}\n')

    print(f"Downloaded transcript for {id} to {filename}")
    return

In [84]:
for video in video_links:
    id = video['link'].split("v=")[1]
    name = video['name']
    file_path = f"{TRANSCRIPT_DIR}/{name}.txt"
    download_transcript(id, file_path, overwrite=False)


Downloaded transcript for ICnFtfN-sUc to transcripts/twitch.txt
Downloaded transcript for Yqasp_4x6zw to transcripts/mongodb.txt
Downloaded transcript for rsa1_d8Z0Zc to transcripts/langchain.txt
Downloaded transcript for HW5Yeqn9eV4 to transcripts/notion.txt
Downloaded transcript for ky2rNqi3BX8 to transcripts/angellist.txt
Downloaded transcript for 70PgHmZ1Aok to transcripts/flexport.txt
Downloaded transcript for 1gLHnRDiIms to transcripts/pinecone.txt


In [78]:
AUDIO_DIR = "audio"
file_name = "test"
url
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'outtmpl': f'{AUDIO_DIR}/{file_name}.mp3',
    'noplaylist': True,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=ZrQJaLemaoE
[youtube] ZrQJaLemaoE: Downloading webpage


[youtube] ZrQJaLemaoE: Downloading ios player API JSON


[youtube] ZrQJaLemaoE: Downloading android player API JSON


[youtube] ZrQJaLemaoE: Downloading iframe API JS


[youtube] ZrQJaLemaoE: Downloading web player API JSON


ERROR: [youtube] ZrQJaLemaoE: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


DownloadError: ERROR: [youtube] ZrQJaLemaoE: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U